In [1]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(
    file_path='./endpoints.json',
    jq_schema='.',
    text_content=False)

data = loader.load()

data

[Document(page_content='{"FKB Tiltak (DEMO)": {"type": "application/json", "href": "https://alenos-tester001.azurewebsites.net/collections?f=json", "description": "A API endpoint with JSON response which conforms to the OGC API Features standard. This is a it contains a list of collections.", "links": [{"type": "application/yaml", "title": "Schema definition for Collections in OGC API Features", "href": "https://schemas.opengis.net/ogcapi/features/part1/1.0/openapi/schemas/collections.yaml"}]}}', metadata={'source': '/home/oskar/dev/prosjektoppgave/src/python/examples/ogc_api_features_analysis/endpoints.json', 'seq_num': 1})]

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
from langchain.agents import tool
import requests

@tool
def make_http_request(url: str) -> str:
    """Makes a HTTP reuqest to the specified URL, and returns the result as a string."""

    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text  
        else:
            return f"Request failed with status code: {response.status_code}"
    except requests.RequestException as e:
        return f"An error occurred: {e}"
    
tools = [make_http_request]

In [4]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant GIS assistant.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [5]:
from langchain.tools.render import format_tool_to_openai_function

llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

In [6]:
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({"input": "explain the content at https://alenos-tester001.azurewebsites.net/collections?f=json"})



> Entering new AgentExecutor chain...

Invoking: `make_http_request` with `{'url': 'https://alenos-tester001.azurewebsites.net/collections?f=json'}`


{
    "collections": [
        {
            "id": "tiltak",
            "title": "FKB Tiltak (DEMO)",
            "description": "FKB Tiltak fra Kristiansand Kommune (DEMO)",
            "keywords": [
                "monitoring"
            ],
            "links": [
                {
                    "type": "text/html",
                    "rel": "canonical",
                    "title": "information",
                    "href": "http://www.naturalearthdata.com/",
                    "hreflang": "en-US"
                },
                {
                    "type": "application/json",
                    "rel": "root",
                    "title": "The landing page of this server as JSON",
                    "href": "https://alenos-tester001.azurewebsites.net?f=json"
                },
                {
                    "t

{'input': 'explain the content at https://alenos-tester001.azurewebsites.net/collections?f=json',
 'output': 'The content at https://alenos-tester001.azurewebsites.net/collections?f=json is a JSON document that contains information about different collections. Each collection has the following properties:\n\n1. id: The unique identifier of the collection.\n2. title: The title or name of the collection.\n3. description: A brief description of the collection.\n4. keywords: An array of keywords associated with the collection.\n5. links: An array of links related to the collection. Each link has the following properties:\n   - type: The type of the link.\n   - rel: The relationship of the link to the collection.\n   - title: The title or description of the link.\n   - href: The URL of the link.\n6. extent: The spatial and temporal extent of the collection. The spatial extent is defined by a bounding box (bbox) and a coordinate reference system (crs). The temporal extent is defined by an in